In [1]:
import os
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, GenerationConfig
from dotenv import load_dotenv
from huggingface_hub import login 
from deploy_utils import fit_into_model, gen_prompt, get_starter_prompt, check_gpu, get_model_tokenizer
load_dotenv("/mnt/c/projects/game/token.env")

# Retrieve the API token
api_token = os.getenv("api_token")

# Log in to the Hugging Face Hub
login(token = api_token)

# Check if CUDA is available
check_gpu()

Number of GPUs available: 1
GPU 0: NVIDIA GeForce RTX 3060


In [1]:
# Define the model name
model, tokenizer = get_model_tokenizer("google/gemma-7b-it")

NameError: name 'get_model_tokenizer' is not defined

In [1]:
import sys
sys.path.insert(0, '..')

from retrival.helper_functions import get_embeddings, create_vector_db, similarity_search

In [3]:
docs, embeddings = get_embeddings()
db = create_vector_db(docs, embeddings, 'instructions')

Created a chunk of size 214, which is longer than the specified 100
Created a chunk of size 430, which is longer than the specified 100
Created a chunk of size 311, which is longer than the specified 100
Created a chunk of size 924, which is longer than the specified 100
Created a chunk of size 328, which is longer than the specified 100


In [5]:
query = "I cut the beasts head off"
query_docs = db.similarity_search(query, k = 1)
print(query_docs[0].page_content)

[Monster Encounter/Combat Mechanics/Fighting With Monster/Kill beast]
- When the player encounters a monster, describe its appearance, behavior, and any distinguishing features.
- Instruct the player to type "Attack" followed by the monster's name to engage in combat.
- Alternatively, the player can type "Pet" followed by the monster's name to attempt to befriend the monster.
- If the player chooses to attack, resolve the combat encounter based on the player's actions and the monster's behavior.
- If the player chooses to pet the monster, describe the player's approach and the monster's reaction.
- The monster may respond with either aggression or friendliness, depending on its nature and the player's actions.
- Provide feedback on the outcome of the interaction, including any consequences for the player's choice.
- The player can target/cut the monster's head or eye or heart to kill it or make it barely alive.


In [4]:
prompt = get_starter_prompt(tokenizer)
for i in range(0,100):
    output = fit_into_model(prompt, tokenizer, model)
    print(output)
    user_input = input('What is your next move?')
    prompt = gen_prompt(user_input,output, model)

KeyboardInterrupt: 